In [39]:
##  Загрузка библиотек
import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [40]:
## Читаем Датасет  
df = pandas.read_csv('gamesPC.csv')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [42]:
df.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [43]:
##  Кодировка в числа
# Присваиваем каждому уникальному элементу таблицы его уникальный номер 
categorical_columns = df.select_dtypes(include='object').columns
encoder = preprocessing.LabelEncoder()

le = LabelEncoder()
df['Name'] = encoder.fit_transform(df['Name'])
df['Platform'] = encoder.fit_transform(df['Platform'])
df['Genre'] = encoder.fit_transform(df['Genre'])
df['Publisher'] = df['Publisher'].apply(str)
df['Publisher'] = encoder.fit_transform(df['Publisher'])
df.head(5)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,11007,26,2006.0,10,359,41.49,29.02,3.77,8.46,82.74
1,2,9327,11,1985.0,4,359,29.08,3.58,6.81,0.77,40.24
2,3,5573,26,2008.0,6,359,15.85,12.88,3.79,3.31,35.82
3,4,11009,26,2009.0,10,359,15.75,11.01,3.28,2.96,33.00
4,5,7346,5,1996.0,7,359,11.27,8.89,10.22,1.00,31.37


In [44]:
##  иницилизируем df1 для прогноза
df1 = df
df1.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,11007,26,2006.0,10,359,41.49,29.02,3.77,8.46,82.74
1,2,9327,11,1985.0,4,359,29.08,3.58,6.81,0.77,40.24
2,3,5573,26,2008.0,6,359,15.85,12.88,3.79,3.31,35.82
3,4,11009,26,2009.0,10,359,15.75,11.01,3.28,2.96,33.00
4,5,7346,5,1996.0,7,359,11.27,8.89,10.22,1.00,31.37


In [45]:
##  Избавляемся от NaN и заполняем средними
df1['Year'] = df['Year'].astype(float)
df1 = df1.fillna(0) ## Избавляется от NaN
df1['Year'].replace(0,df1['Year'].mean(axis=0),inplace=True) ## Заполняет нули Средникми значениями
##
df1['Year'] = df1.Year.astype(int) ## переводим в int 
##
df1.head(5)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,11007,26,2006,10,359,41.49,29.02,3.77,8.46,82.74
1,2,9327,11,1985,4,359,29.08,3.58,6.81,0.77,40.24
2,3,5573,26,2008,6,359,15.85,12.88,3.79,3.31,35.82
3,4,11009,26,2009,10,359,15.75,11.01,3.28,2.96,33.00
4,5,7346,5,1996,7,359,11.27,8.89,10.22,1.00,31.37


In [46]:
## Для успешного обучения модели разбрасаем DF в случайном порядке 
from sklearn.utils import shuffle  
df1 = shuffle(df1) 
df1

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
16134,16137,4889,4,2008,3,64,0.00,0.00,0.01,0.00,0.01
11719,11721,2282,13,2009,7,112,0.01,0.05,0.00,0.01,0.08
4936,4938,1933,13,2009,9,525,0.00,0.32,0.00,0.07,0.39
12085,12087,6993,15,1998,0,151,0.00,0.00,0.06,0.00,0.07
6312,6314,6130,28,2011,10,525,0.14,0.10,0.00,0.03,0.27
...,...,...,...,...,...,...,...,...,...,...,...
14409,14412,9239,16,2009,2,11,0.00,0.00,0.03,0.00,0.03
16090,16093,4160,4,2010,6,471,0.01,0.00,0.00,0.00,0.01
5576,5578,6333,19,2009,10,494,0.30,0.00,0.00,0.03,0.32
1219,1221,6269,26,2008,9,138,0.71,0.65,0.01,0.17,1.53


In [50]:
#Случайный лес — популярный алгоритм контролируемого обучения, заключающийся в 
#использовании комитета (ансамбля) решающих деревьев. «Ансамбль» означает, что он берет кучу 
#«слабых учеников» и объединяет их, чтобы сформировать один сильный предиктор. 
#«Слабые ученики» — это все случайные реализации деревьев решений, которые объединяются для формирования
#сильного предсказателя — случайного леса.
#
#X,Y - это вектора для обученяия модели, где X - первичные данные, Y - данные, которые мы попытаемся спрогнозировать
## Тренируем модель для прогнозирования Продаж на территории Америке
forestNA = RandomForestRegressor()
X = df1.values[:, 0:6]
Y = df1.values[:, 6]
trainX, testX, trainY, testY = train_test_split( X, Y, test_size = 0.3)
forestNA.fit(trainX, trainY)
print('Accuracy: \n', forestNA.score(testX, testY))
pred = forestNA.predict(testX)
## Результат показывает, что модель обучилась с хорошим успехом ~90% 
## Это результат сравнения выборочной и тестовой части Датасета 
## В pred хранятся Данные, которые выдумала наша модель 

Accuracy: 
 0.8887802230979039


array([0.049 , 0.0868, 0.0382, ..., 0.0122, 0.0834, 0.0022])

In [62]:
## Тренируем модель для прогнозирования Продаж на территории Европы
forestEU = RandomForestRegressor()
X = df1.values[:, 0:6]
Y = df1.values[:, 7]
trainX, testX, trainY, testY = train_test_split( X, Y, test_size = 0.3)
forestEU.fit(trainX, trainY)
print('Accuracy: \n', forestEU.score(testX, testY))
pred = forestEU.predict(testX)
## Результат показывает, что модель обучилась с хорошим успехом около 90% 
## Это результат сравнения выборочной и тестовой части Датасета 
## В pred хранятся Данные, которые выдумала наша модель 

Accuracy: 
 0.8339882712726385


In [61]:
## Тренируем модель для прогнозирования Продаж на территории Японии
forestJP = RandomForestRegressor()
X = df1.values[:, 0:6]
Y = df1.values[:, 8]
trainX, testX, trainY, testY = train_test_split( X, Y, test_size = 0.3)
forestJP.fit(trainX, trainY)
print('Accuracy: \n', forestJP.score(testX, testY))
pred = forestJP.predict(testX)
## Результат показывает, что модель обучилась с хорошим успехом около 80% 
## Это результат сравнения выборочной и тестовой части Датасета 
## В pred хранятся Данные, которые выдумала наша модель 

Accuracy: 
 0.7550672722999534


In [60]:
## Тренируем модель для прогнозирования Продаж на территории Остального мира
forestOther = RandomForestRegressor()
X = df1.values[:, 0:6]
Y = df1.values[:, 9]
trainX, testX, trainY, testY = train_test_split( X, Y, test_size = 0.3)
forestOther.fit(trainX, trainY)
print('Accuracy: \n', forestOther.score(testX, testY))
pred = forestOther.predict(testX)
## Результат показывает, что модель обучилась с перемнным успехом около 60%
## Это результат сравнения выборочной и тестовой части Датасета 
## В pred хранятся Данные, которые выдумала наша модель 

Accuracy: 
 0.6302605526150784


In [65]:
forestNA = RandomForestRegressor()
X = df1.values[:, 0:6]
Y = df1.values[:, 6]
trainX, testX, trainY, testY = train_test_split( X, Y, test_size = 0.3)
forestNA.fit(trainX, trainY)
print('Accuracy: \n', forestNA.score(testX, testY))
pred = forestNA.predict(testX)
## Результат показывает, что модель обучилась с хорошим успехом ~90% 
## Это результат сравнения выборочной и тестовой части Датасета 
## В pred хранятся Данные, которые выдумала наша модель 

Accuracy: 
 0.7838483399632507
